In [5]:
import cv2
import numpy as np
import os
import time

In [21]:
class CaptureManager(object):
    def __init__(self,cature,previewWindowMananger=None,shouldMirrorPreview=False):
        self.previewWindowMananger=previewWindowMananger
        self.shouldMirrorPreview=shouldMirrorPreview
        
        self._capture=cature
        self._channel=0
        self._enteredFrame=False
        self._frame=None
        self._imageFileName=None
        self._videoFileNmae=None
        self._videoEncoding=None
        self._videoWriter=None
        
        self._startTime=None
        self._framesElapsed=int(0)
        self._fpsEstimate=None
        
    @property
    def channel(self):
        return self._channel
    
    @channel.setter   
    def channel(self,value):
        if self._channel != value:
            self._channel = value
            self._frame = None
    
    @property
    def frame(self):
        if self._enteredFrame and self._frame is None:
            _,self._frame=self._capture.retrieve()
            return self._frame
        
    @property
    def isWritingImage(self):
        return self._imageFileName is not None
    
    @property 
    def isWritingVideo(self):
        return self._videoFileNmae is not None
    
    def enterFrame(self):
        assert not self._enteredFrame,'以前的enterFrame没有匹配的输出'
        if self._capture is not None:
            self._enteredFrame = self._capture.grab()
            
    def exitFrame(self):
        if self.frame is None:
            self._enteredFrame=False
            return 
        
        if self._framesElapsed == 0:
            self._startTime = time.time()
        else:
            timeElapsed = time.time() - self._startTime
            self._fpsEstimate = self._framesElapsed/timeElapsed
        self._framesElapsed+=1
        
        if self.previewWindowMananger is not None:
            if self.shouldMirrorPreview:
                mirroredFrame = np.fliplr(self._frame).copy()
                self.previewWindowMananger.show(mirroredFrame)
            else:
                self.previewWindowMananger.show(self._frame)
                
        if self.isWritingImage:
            cv2.imwrite(self._imageFileName,self._frame)
            self._imageFileName=None
            
        self._writeVideoFrame()
        
        self._frame=None
        self._enteredFrame=False
        
    def writeImage(self,filename):
        self._imageFileName=file
        
    def startWritingVideo(self,filename,encoding=cv2.VideoWriter_fourcc('I','4','2','0')):
        self._videoFileNmae = filename
        self._videoEncoding = encoding
        
    def stopWritingVideo(self):
        self._videoFileNmae = None
        self._videoEncoding = None
        self._videoWriter = None
        
    def _writeVideoFrame(self):
        if not self.isWritingVideo:
            return
        
        if self._videoWriter is None:
            fps=self._capture.get(cv2.CAP_PROP_FPS)
            if fps == 0.0:
                if self._framesElapsed < 20:
                    return
                else:
                    fps=self._fpsEstimate
            size=(int(self._capture.get(cv2.CAP_PROP_FRAME_WIDTH)),int(self._capture.get(cv2.CAP_PROP_FRAME_HEIGHT)))
            self._videoWriter=cv2.VideoWriter(self._videoFileNmae,self,_videoEncoding,fps,size)
        self._videoWriter.write(self._frame)

In [11]:
class WindowManager(object):
    def __init__(self,windowName,keypressCallback=None):
        self.keypressCallback=keypressCallback
        self._windowName=windowName
        self._isWindowCreate=False
        
    @property
    def isWindowCreated(self):
        return self._isWindowCreate
    
    def createWindow(self):
        cv2.namedWindow(self._windowName)
        self._isWindowCreate=True
        
    def show(self,frame):
        cv2.imshow(self._windowName,frame)
        
    def destroyWindow(self):
        cv2.destroyWindow(self._windowName)
        self._isWindowCreate =False
    
    def processEvents(self):
        keycode = cv2.waitKey(1)
        if self.keypressCallback is not None and keycode != -1:
            keycode&=0xFF
            self.keypressCallback(keycode)

In [22]:
class Cameo(object):
    def __init__(self):
        self._windowManager=WindowManager('cameo',self.onkeypress)
        self._captureManager=CaptureManager(cv2.VideoCapture(0),self._windowManager,True)
        
    def run(self):
        self._windowManager.createWindow()
        while self._windowManager.isWindowCreated:
            self._captureManager.enterFrame()
            frame=self._captureManager.frame
  
    def onkeypress(self,keycode):
        if keycode == 32:
            self._captureManager.writeImage('screenShot.png')
        elif keycode == 9:
            if not self._captureManager.isWritingVideo:
                self._captureManager.startWritingVideo('screencast.avi')
            else:
                self._captureManager.stopWritingVideo()
        elif keycode == 27:
            self._windowManager.destroyWindow()

if __name__ == '__main__':
    Cameo().run()

AssertionError: 以前的enterFrame没有匹配的输出

In [1]:
import cv2
import numpy as np
import time


'''
    视频管理
'''
class CaptureManager(object):
    def __init__(self,
                 capture, #摄像头通道
                 previewWindowManager=None,#窗口管理器
                 shouldMirrorPreview=False):#摄像头预览的镜像选项

        self.previewWindowManager=previewWindowManager
        self.shouldMirrorPreview=shouldMirrorPreview

        #定义非公有变量,单下划线开始,为保护变量,只有类对象或子类对象可以访问 protected
        #如果以双下划线开始,为私有成员变量,只有类对象自已可以访问,像private
        self._capture=capture
        self._channel=0
        self._enteredFrame=False
        self._frame=None
        self._imageFilename=None
        self._videoFilename=None
        self._videoEncoding=None
        self._videoWriter=None

        self.startTime=None
        self._framesElapsed=int(0)
        self._fpsEstimate=None

    @property
    def channel(self):
        return self._channel

    @channel.setter
    def channel(self,value):
        if self._channel!=value:
            self._channel=value
            self._frame=None

    @property
    def frame(self):
        if self._enteredFrame and self._frame is None:
            _,self._frame=self._capture.retrieve()
        return self._frame

    @property
    def isWritingImage(self):
        return self._imageFilename is not None

    @property
    def isWritingVideo(self):
        return self._videoFilename is not None


    #只能同步一帧
    def enterFrame(self):

        assert not self._enteredFrame, \
            'previous enterFrame() had no matching exitFrame()'

        if self._capture is not None:
            self._enteredFrame=self._capture.grab()

    #可以从当前通道中取得图像,估计帧率,显示图像,执行暂停的请求,向文件中写入图像
    def exitFrame(self):
        if self.frame is None:
            self._enteredFrame=False
            return

        #计算帧率
        if self._framesElapsed==0:
            self._startTime=time.time()
        else:
            timeElapsed=time.time()-self._startTime
            self._fpsEstimate=self._framesElapsed/timeElapsed

        self._framesElapsed+=1

        #通过窗体显示图像
        if self.previewWindowManager is not None:
            if self.shouldMirrorPreview:
                mirroredFrame=np.fliplr(self._frame).copy()
                self.previewWindowManager.show(mirroredFrame)
            else:
                self.previewWindowManager.show(self._frame)

        #保存图像文件
        if self.isWritingImage:
            cv2.imwrite(self._imageFilename,self._frame)
            self._imageFilename=None

        #保存视频文件
        self._writeVideoFrame()

        #释放资源
        self._frame=None
        self._enteredFrame=False

    #保存图片,公有函数
    def writeImage(self,filename):
        self._imageFilename=filename

    #开始保存视频,公有函数
    def startWritingVideo(self,filename,encoding=cv2.VideoWriter_fourcc('I','4','2','0')):
        self._videoFilename=filename
        self._videoEncoding=encoding

    #停止视频写入,公有函数
    def stopWritingVideo(self):
        self._videoFilename=None
        self._videoEncoding=None
        self._videoWriter=None

    #写入视频帧
    def _writeVideoFrame(self):
        if not self.isWritingVideo:
            return

        if self._videoWriter is None:
            fps=self._capture.get(cv2.CAP_PROP_FPS)
            if fps==0.0:
                if self._framesElapsed<20:
                    return
                else:
                    fps=self._fpsEstimate
            size=(int(self._capture.get(cv2.CAP_PROP_FRAME_WIDTH)),
                  int(self._capture.get(cv2.CAP_PROP_FRAME_HEIGHT)))

            self._videoWriter=cv2.VideoWriter(self._videoFilename,
                                              self._videoEncoding,
                                              fps,
                                              size)
        self._videoWriter.write(self._frame)

'''
    窗口管理,支持键盘事件
'''
class WindowManager(object):

    def __init__(self,
                 windowName,#窗体名称
                 keypressCallback=None):#按键回调函数
        self.keypressCallback=keypressCallback

        self._windowName=windowName
        self._isWindowCreate=False

    #检查窗体是否被创建
    @property
    def isWindowCreated(self):
        return self._isWindowCreate

    #创建窗体
    def createWindow(self):
        cv2.namedWindow(self._windowName)
        self._isWindowCreate=True

    #显示图像
    def show(self,frame):
        cv2.imshow(self._windowName,frame)

    #关闭窗体释放资源
    def destroyWindow(self):
        cv2.destroyWindow(self._windowName)
        self._isWindowCreate=False

    #处理键盘事件
    def processEvents(self):
        keycode=cv2.waitKey(1)
        if self.keypressCallback is not None and keycode!=-1:
            keycode&=0xFF #ESC 退出
            self.keypressCallback(keycode)

In [2]:
class Cameo(object):

    def __init__(self):
        
        self._windowManager=WindowManager('Cameo',self.onkeypress)

        self._captureManager=CaptureManager(cv2.VideoCapture(0),self._windowManager,True)

    def run(self):
        self._windowManager.createWindow()
        while self._windowManager.isWindowCreated:
            self._captureManager.enterFrame()
            frame=self._captureManager.frame

            self._captureManager.exitFrame()
            self._windowManager.processEvents()

    def onkeypress(self,keycode):
        '''
            space-> 载图
            tab->启动和停止视频录制
            esc->退出应用

        :param keycode:
        :return:
        '''
        if keycode==32:#space
            self._captureManager.writeImage('screenshot.png')
        elif keycode==9:#tab
            if not self._captureManager.isWritingVideo:
                self._captureManager.startWritingVideo('screencast.avi')
            else:
                self._captureManager.stopWritingVideo()
        elif keycode==27:#esc
            self._windowManager.destroyWindow()
if __name__=='__main__':
    Cameo().run()

In [7]:
import cv2
import numpy as np

# 基本的运动检测
camera = cv2.VideoCapture(0)
es = cv2 .getStructuringElement(cv2.MORPH_ELLIPSE,(9,4))
kernel = np.ones((5,5),np.uint8)
background = None

while(True):
    ret,frame =  camera.read()
    if background is None:
        background = cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
        background = cv2.GaussianBlur(background,(21,21),0)
        continue
        
    gray_frame = cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
    gray_frame = cv2.GaussianBlur(gray_frame,(21,21),0)
    
    diff = cv2.absdiff(background,gray_frame)
    diff = cv2.threshold(diff,25,255,cv2.THRESH_BINARY)[1]
    diff = cv2.dilate(diff,es,iterations = 2)
    image,cnts,hierarchy = cv2.findContours(diff.copy(),cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
    
    for c in cnts:
        if cv2.contourArea(c) < 1500:
            continue
        (x,y,w,h) = cv2.boundingRect(c)
        cv2.rectangle(frame,(x,y),(x+w,y+h),(0,255,0),2)
    cv2.imshow('contours',frame)
    cv2.imshow('dif',diff)
    
    if cv2.waitKey(int(1000/12)) & 0xff == ord('q'):
        break
        
    cv2.destroyAllWindows()
    camera.release()

error: OpenCV(3.4.1) C:\projects\opencv-python\opencv\modules\imgproc\src\color.cpp:11147: error: (-215) scn == 3 || scn == 4 in function cv::cvtColor


In [1]:
import cv2
import numpy as np

# 捕获摄像头图像
camera = cv2.VideoCapture(0)
#
es = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (9, 4))
kernel = np.ones((5, 5), np.uint8)
background = None

while (True):
    ret, frame = camera.read()
    # 将第一帧设为图像的背景
    if background is None:
        # 转换颜色空间
        background = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        # 高斯模糊
        background = cv2.GaussianBlur(background, (21, 21), 0)
        continue
    # 转换颜色空间并作模糊处理
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    gray_frame = cv2.GaussianBlur(gray_frame, (21, 21), 0)

    # 取得差分图
    diff = cv2.absdiff(background, gray_frame)
    diff = cv2.threshold(diff, 25, 255, cv2.THRESH_BINARY)[1]
    # 膨胀
    diff = cv2.dilate(diff, es, iterations=2)

    # 得到图像中目标的轮廓
    image, cnts, hierarchy = cv2.findContours(diff.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    for c in cnts:
        if cv2.contourArea(c) < 1500:
            continue
        # 计算矩形边框
        (x, y, w, h) = cv2.boundingRect(c)
        # 绘制矩形
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
    # 显示图像
    cv2.imshow('contours', frame)
    cv2.imshow('dif', diff)
    if cv2.waitKey(int(1000 / 12)) & 0xFF == ord('q'):
        break
cv2.destroyAllWindows()
camera.release()

In [1]:
import cv2
cap = cv2.VideoCapture(0)
ret,frame = cap.read()
r,h,c,w = 10,200,10,200
track_window = (c,r,w,h)


In [3]:
# 卡尔曼滤波器
import cv2
import numpy as np

frame = np.zeros((800,800,3),np.uint8)
last_measurement = current_measurement = np.array((2,1),np.float32)
last_prediction = current_prediction = np.zeros((2,1),np.float32)

def mousemove(event,x,y,s,p):
    global frame,current_measurement,measurements,last_measurement,current_prediction,last_prediction
    last_prediction = current_prediction
    last_measurement = current_measurement
    current_measurement = np.array([[np.float32(x)],[np.float32(y)]])
    kalman.correct(current_measurement)
    
    current_measurement = kalman.predict()
    lmx,lmy = last_measurement[0],last_measurement[1]
    cmx,cmy = current_measurement[0],current_measurement[1]
    lpx,lpy = last_prediction[0],last_prediction[1]
    cpx,cpy = current_prediction[0],current_prediction[1]
    
    cv2.line(frame,(lmy,lmy),(cmx,cmy),(0,100,0))
    cv2.line(frame,(lpx,lpy),(cpx,cpy),(0,0,200))

cv2.namedWindow('kalman_tracker')
cv2.setMouseCallback('kalman_tracker',mousemove)

kalman = cv2.KalmanFilter(4,2)
kalman.measurementMatrix = np.array([[1,0,0,0],[0,1,0,0]],np.float32)
kalman.transitionMatrix = np.array([[1,0,1,0],[0,1,0,1],[0,0,1,0],[0,0,0,1]],np.float32)
kalman.processNoiseCov = np.array([[1,0,0,0],[0,1,0,0],[0,0,1,0],[0,0,0,1]],np.float32)

while True:
    cv2.imshow('Kalman_tracker',frame)
    if cv2.waitKey(3000) & 0xff == 27:
        break
cv2.destroyAllWindows()